In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN','NAME'])

In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = application_df['APPLICATION_TYPE'].value_counts().loc[lambda x: x<500].index

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = application_df['CLASSIFICATION'].value_counts().loc[lambda x: x<1000].index

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

In [5]:
# drop income amount column due to it high variablity we will use it in another model
catagorical_cols = application_df[['APPLICATION_TYPE',
                                   'CLASSIFICATION',
                                   'AFFILIATION',
                                   'USE_CASE',
                                   'ORGANIZATION','STATUS','INCOME_AMT','SPECIAL_CONSIDERATIONS']]

In [ ]:
# get dummies for catagorical variables
application_dummies = pd.get_dummies(catagorical_cols, dtype=int)
application_dummies.head()

In [ ]:
# Merge dummy catagorical columns onto dataframe to create a scaled dataset and drop old columns
# create copy of original df
scaled_application_df = application_df.copy()
# combine the data frame with dummy data using concat
scaled_application_df = pd.concat([scaled_application_df, application_dummies], axis=1)

# drop catagorical columns
scaled_application_df = scaled_application_df.drop(columns=['APPLICATION_TYPE',
                                                            'CLASSIFICATION',
                                                            'AFFILIATION',
                                                            'USE_CASE',
                                                            'ORGANIZATION','STATUS','INCOME_AMT','SPECIAL_CONSIDERATIONS'])
# preview dataframe
scaled_application_df.head()

In [8]:
# Split our preprocessed data into our features and target arrays
y = scaled_application_df['IS_SUCCESSFUL']
X = scaled_application_df.drop(columns='IS_SUCCESSFUL')
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [9]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Lets first try and optimize by adding another hidden layer

nn_extra_layer = tf.keras.models.Sequential()

# First hidden layer
nn_extra_layer.add(tf.keras.layers.Dense(units=16, activation='relu',input_dim=42))

# Second hidden layer
nn_extra_layer.add(tf.keras.layers.Dense(units=16, activation='relu'))

# Third hidden layer with less units
nn_extra_layer.add(tf.keras.layers.Dense(units=8, activation='relu'))


# Output layer
nn_extra_layer.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn_extra_layer.summary()

In [11]:
# compile model using same optimizer
nn_extra_layer.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# fit the model
fit_model = nn_extra_layer.fit(X_train_scaled, y_train, epochs=100)


In [ ]:
model_loss, model_accuracy = nn_extra_layer.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

accuracy decreased when adding another hidden layer... back tho the drawing board

In [ ]:
# lets try and add more things to bins since there were some outliers
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
# since there is a large outlier in T3 applications we are going to isolate that value and use all other types as other
application_types_to_replace = application_df['APPLICATION_TYPE'].value_counts().loc[lambda x: x<2000].index

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
# lets do something similar with classification because the C1000 is so much higher than the rest
classifications_to_replace = application_df['CLASSIFICATION'].value_counts().loc[lambda x: x<10000].index

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

In [16]:

# Create list of catagorical values for easier conversion
catagorical_cols = application_df[['APPLICATION_TYPE',
                                   'CLASSIFICATION',
                                   'AFFILIATION',
                                   'USE_CASE',
                                   'ORGANIZATION','STATUS','INCOME_AMT','SPECIAL_CONSIDERATIONS']]

In [ ]:
# get dummies for catagorical variables
application_dummies = pd.get_dummies(catagorical_cols, dtype=int)
application_dummies.head()

In [ ]:
# Merge dummy catagorical columns onto dataframe to create a scaled dataset and drop old columns
# create copy of original df
scaled_application_df = application_df.copy()
# combine the data frame with dummy data using concat
scaled_application_df = pd.concat([scaled_application_df, application_dummies], axis=1)

# drop catagorical columns
scaled_application_df = scaled_application_df.drop(columns=['APPLICATION_TYPE',
                                                            'CLASSIFICATION',
                                                            'AFFILIATION',
                                                            'USE_CASE',
                                                            'ORGANIZATION','STATUS','INCOME_AMT','SPECIAL_CONSIDERATIONS'])
# preview dataframe
scaled_application_df.head()

In [19]:
# Split our preprocessed data into our features and target arrays
y = scaled_application_df['IS_SUCCESSFUL']
X = scaled_application_df.drop(columns='IS_SUCCESSFUL')
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [20]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn_bins = tf.keras.models.Sequential()

# First hidden layer
nn_bins.add(tf.keras.layers.Dense(units=16, activation='relu',input_dim=31))

# Second hidden layer
nn_bins.add(tf.keras.layers.Dense(units=16, activation='relu'))

# Output layer
nn_bins.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn_bins.summary()

In [22]:
# compile model using same optimizer
nn_bins.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# fit the model
fit_model = nn_bins.fit(X_train_scaled, y_train, epochs=100)

In [24]:
model_loss, model_accuracy = nn_bins.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5886 - accuracy: 0.7082 - 498ms/epoch - 2ms/step
Loss: 0.5886472463607788, Accuracy: 0.7082215547561646


accuracy decreased a lot... Lets change a bunch of things one more time and see what happens

In [36]:
# Reread csv
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [37]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN','NAME'])

In [38]:
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [39]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
# since when we reduced the bins and increased the variables in them I will revert to a lower cutoff
application_types_to_replace = application_df['APPLICATION_TYPE'].value_counts().loc[lambda x: x<50].index

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
T13         66
Other       54
Name: count, dtype: int64

In [40]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
# lets do something similar with classification with a lower cutoff to have mroe bins
classifications_to_replace = application_df['CLASSIFICATION'].value_counts().loc[lambda x: x<100].index

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: count, dtype: int64

In [41]:

# Create list of catagorical values for easier conversion
catagorical_cols = application_df[['APPLICATION_TYPE',
                                   'CLASSIFICATION',
                                   'AFFILIATION',
                                   'USE_CASE',
                                   'ORGANIZATION','STATUS','INCOME_AMT','SPECIAL_CONSIDERATIONS']]

In [42]:
# Convert categorical data to numeric with `pd.get_dummies`
application_dummies = pd.get_dummies(catagorical_cols, dtype=int)
application_dummies.head()

,STATUS,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T13,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [43]:
# Merge dummy catagorical columns onto dataframe to create a scaled dataset and drop old columns
# create copy of original df
scaled_application_df = application_df.copy()
# combine the data frame with dummy data using concat
scaled_application_df = pd.concat([scaled_application_df, application_dummies], axis=1)

# drop catagorical columns
scaled_application_df = scaled_application_df.drop(columns=['APPLICATION_TYPE',
                                                            'CLASSIFICATION',
                                                            'AFFILIATION',
                                                            'USE_CASE',
                                                            'ORGANIZATION','STATUS','INCOME_AMT','SPECIAL_CONSIDERATIONS'])
# preview dataframe
scaled_application_df.head()

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T13,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,5000,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,108590,1,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,5000,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,6692,1,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,142590,1,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [44]:
# Split our preprocessed data into our features and target arrays
y = scaled_application_df['IS_SUCCESSFUL']
X = scaled_application_df.drop(columns='IS_SUCCESSFUL')
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [45]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [50]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn_changes = tf.keras.models.Sequential()

# First hidden layer
nn_changes.add(tf.keras.layers.Dense(units=16, activation='relu',input_dim=50))

# Second hidden layer with less neurons
nn_changes.add(tf.keras.layers.Dense(units=10, activation='relu'))

# Output layer
nn_changes.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn_changes.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_13 (Dense)            (None, 16)                816       
                                                                 
 dense_14 (Dense)            (None, 10)                170       
                                                                 
 dense_15 (Dense)            (None, 1)                 11        
                                                                 
Total params: 997 (3.89 KB)
Trainable params: 997 (3.89 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [51]:
# compile the model
# adam is widely used and works well so lets stick with adam
nn_changes.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model
fit_model = nn_changes.fit(X_train_scaled, y_train, epochs=100)

In [53]:
model_loss, model_accuracy = nn_changes.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5530 - accuracy: 0.7313 - 970ms/epoch - 4ms/step
Loss: 0.5530339479446411, Accuracy: 0.7313119769096375


accuracy increased from our original model! But not to greater than 75%